In [2]:
# 导入PaddlePaddle
import paddle
from paddle import fluid

# 导入conll数据集
from paddle.dataset import conll05

In [10]:
# 字典信息
word_dict, _, label_dict=conll05.get_dict()

In [11]:
# 设置超参数
WORD_DIM=32 # 设置词向量大小
BATCH_SIZE=10 # 训练批数据大小
EPOCH_NUM=20 # 训练轮数
HIDDEN_DIM=512 # 模型隐层大小
LEARNING_RATE=1e-1 # 学习率大小

In [13]:
# 设置输入word和目标label的变量
word=fluid.layers.data(name="word_data",shape=[1],dtype='int64',lod_level=1)
target=fluid.layers.data(name="target",shape=[1],dtype='int64',lod_level=1)

# 将字用embedding标示，并通过线性层
embedding=fluid.layers.embedding(size=[len(word_dict),WORD_DIM],input=word,
                                param_attr=fluid.ParamAttr(name="emb",trainable=False))
hidden_0=fluid.layers.fc(input=embedding,size=HIDDEN_DIM,act="tanh")


# 用 RNNs 得到输入的提取特征并做变换.
hidden_1 = fluid.layers.dynamic_lstm(
    input=hidden_0, size=HIDDEN_DIM,
    gate_activation='sigmoid',
    candidate_activation='relu',
    cell_activation='sigmoid')
feature_out = fluid.layers.fc(input=hidden_1, size=len(label_dict), act='tanh')

In [14]:
# 调用内置 CRF 函数并做状态转换解码.
crf_cost = fluid.layers.linear_chain_crf(
    input=feature_out, label=target,
    param_attr=fluid.ParamAttr(name='crfw', learning_rate=LEARNING_RATE))
avg_cost = fluid.layers.mean(crf_cost)
# 调用 SGD 优化函数并优化平均损失函数.
fluid.optimizer.SGD(learning_rate=LEARNING_RATE).minimize(avg_cost)

([outputs {
    parameter: "Out"
    arguments: "tmp_0"
  }
  type: "fill_constant"
  attrs {
    name: "op_role"
    type: INT
    i: 18
  }
  attrs {
    name: "dtype"
    type: INT
    i: 5
  }
  attrs {
    name: "shape"
    type: LONGS
    longs: 1
  }
  attrs {
    name: "value"
    type: FLOAT
    f: 0.10000000149011612
  }
  attrs {
    name: "op_role_var"
    type: STRINGS
  }
  attrs {
    name: "force_cpu"
    type: BOOLEAN
    b: false
  }
  attrs {
    name: "op_namescope"
    type: STRING
    s: "/optimizer/scale_with_param_lr/"
  }
  attrs {
    name: "op_callstack"
    type: STRINGS
    strings: "  File \"D:\\ProgramData\\Anaconda3\\lib\\site-packages\\paddle\\fluid\\framework.py\", line 1771, in append_op\n    attrs=kwargs.get(\"attrs\", None))\n"
    strings: "  File \"D:\\ProgramData\\Anaconda3\\lib\\site-packages\\paddle\\fluid\\layers\\math_op_patch.py\", line 46, in create_tensor\n    stop_gradient=True)\n"
    strings: "  File \"D:\\ProgramData\\Anaconda3\\lib\\s

In [15]:
# 声明 PaddlePaddle 的计算引擎.
place = fluid.CPUPlace()
exe = fluid.Executor(place)
main_program = fluid.default_main_program()
exe.run(fluid.default_startup_program())

# 由于是 DEMO 因此用测试集训练模型.
feeder = fluid.DataFeeder(feed_list=[word, target], place=place)
shuffle_loader = paddle.reader.shuffle(paddle.dataset.conll05.test(), buf_size=8192)
train_data = paddle.batch(shuffle_loader, batch_size=BATCH_SIZE)




[==================================================]onll05st\conll05st-tests.tar.gz not found, downloading http://paddlemodels.bj.bcebos.com/conll05st/conll05st-tests.tar.gz


In [16]:
# 按 FOR 循环迭代训练模型并打印损失.
batch_id = 0
for pass_id in range(EPOCH_NUM):
    for data in train_data():
        data = [[d[0], d[-1]] for d in data]
        cost = exe.run(main_program, feed=feeder.feed(data), fetch_list=[avg_cost])

        if batch_id % 10 == 0:
            print("avg_cost:\t" + str(cost[0][0]))
        batch_id = batch_id + 1

avg_cost:	130.70784
avg_cost:	66.357544
avg_cost:	65.62456
avg_cost:	64.76676
avg_cost:	58.962116
avg_cost:	55.300606
avg_cost:	44.601475
avg_cost:	50.233833
avg_cost:	39.210854
avg_cost:	42.89013
avg_cost:	46.315186
avg_cost:	38.692986
avg_cost:	29.922
avg_cost:	35.5841
avg_cost:	40.265747
avg_cost:	40.856606
avg_cost:	38.022552
avg_cost:	37.725586
avg_cost:	46.510384
avg_cost:	35.890373
avg_cost:	34.09751
avg_cost:	39.822014
avg_cost:	65.99466
avg_cost:	37.57151
avg_cost:	39.05711
avg_cost:	31.593708
avg_cost:	27.936092
avg_cost:	41.204323
avg_cost:	50.67959
avg_cost:	31.66337
avg_cost:	34.190853
avg_cost:	30.429117
avg_cost:	45.98208
avg_cost:	31.406391
avg_cost:	43.196735
avg_cost:	22.316835
avg_cost:	33.022346
avg_cost:	24.461145
avg_cost:	25.66752
avg_cost:	28.679647
avg_cost:	27.406994
avg_cost:	37.62027
avg_cost:	25.327456
avg_cost:	28.954456
avg_cost:	33.47241
avg_cost:	28.782566
avg_cost:	32.58799
avg_cost:	29.399487
avg_cost:	25.62204
avg_cost:	30.228397
avg_cost:	29.801981


avg_cost:	20.139727
avg_cost:	22.617916
avg_cost:	20.320538
avg_cost:	20.337408
avg_cost:	17.944847
avg_cost:	18.858135
avg_cost:	18.75581
avg_cost:	25.476126
avg_cost:	15.941729
avg_cost:	24.205427
avg_cost:	19.755518
avg_cost:	18.459429
avg_cost:	20.517859
avg_cost:	22.92739
avg_cost:	20.526836
avg_cost:	24.023046
avg_cost:	25.315037
avg_cost:	20.971693
avg_cost:	21.545572
avg_cost:	17.998161
avg_cost:	24.661032
avg_cost:	27.307337
avg_cost:	22.096497
avg_cost:	20.160618
avg_cost:	17.496601
avg_cost:	20.226826
avg_cost:	23.88747
avg_cost:	17.155962
avg_cost:	22.123604
avg_cost:	17.350391
avg_cost:	20.650244
avg_cost:	21.238342
avg_cost:	20.140255
avg_cost:	20.324032
avg_cost:	22.88101
avg_cost:	20.297138
avg_cost:	21.642601
avg_cost:	22.723932
avg_cost:	20.37933
avg_cost:	20.287195
avg_cost:	17.31821
avg_cost:	21.185017
avg_cost:	21.206512
avg_cost:	26.789148
avg_cost:	17.951712
avg_cost:	25.937115
avg_cost:	26.572943
avg_cost:	22.679852
avg_cost:	21.316036
avg_cost:	27.207148
avg_co

avg_cost:	19.761951
avg_cost:	25.309635
avg_cost:	20.614103
avg_cost:	19.434605
avg_cost:	27.51882
avg_cost:	16.40991
avg_cost:	25.840626
avg_cost:	20.331598
avg_cost:	24.408245
avg_cost:	21.463547
avg_cost:	19.873793
avg_cost:	18.02177
avg_cost:	24.788355
avg_cost:	21.466406
avg_cost:	20.404312
avg_cost:	18.804665
avg_cost:	22.39185
avg_cost:	20.133747
avg_cost:	21.808865
avg_cost:	20.782745
avg_cost:	20.397423
avg_cost:	20.895199
avg_cost:	19.571558
avg_cost:	22.853008
avg_cost:	22.320288
avg_cost:	20.451143
avg_cost:	23.430233
avg_cost:	19.121609
avg_cost:	21.618788
avg_cost:	18.559437
avg_cost:	20.817455
avg_cost:	20.756802
avg_cost:	19.813004
avg_cost:	22.391048
avg_cost:	23.595346
avg_cost:	19.695164
avg_cost:	23.732327
avg_cost:	22.088232
avg_cost:	21.138662
avg_cost:	21.438208
avg_cost:	18.155586
avg_cost:	23.7771
avg_cost:	17.533478
avg_cost:	21.582865
avg_cost:	19.518625
avg_cost:	21.88374
avg_cost:	17.192612
avg_cost:	20.84202
avg_cost:	26.329035
avg_cost:	21.232544
avg_cost